In [2]:
%load_ext autoreload
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np

import datetime
from sklearn.grid_search import GridSearchCV

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV


import feature_transformers as ft
from kaggle_helper import KaggleHelper
%aimport feature_transformers
%aimport kaggle_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%aimport

Modules to reload:
feature_transformers kaggle_helper

Modules to skip:



In [7]:
KH = KaggleHelper("pipeline testing.db")

In [4]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
y = raw[['TripType', 'VisitNumber']].groupby('VisitNumber').mean()
X = raw.drop('TripType', axis=1)

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


In [9]:
%autoreload

In [22]:
print(KH.current_run)
KH.start_pipeline()
print(KH.record_metric("validation", "start", "", "", ""))
print(KH.current_run)

None
None
1448480933


In [23]:
%autoreload

group_cols = ['Weekday']
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']

logistic = linear_model.LogisticRegression()
pca = decomposition.PCA()

dfta = ft.DataFrameToArray()
add_returns = ft.NGAddReturns()
gdd = ft.GDeptDescriptionWithTransform(group_cols, keep_cols, [np.sum, np.count_nonzero])
ar1, ar2 = ft.generateMetricTransformPair(KH, "add_returns")

pipe = Pipeline(steps=[
        ('ar1', ar1),
        ('add_returns', add_returns),
        ('ar2', ar2),
        ('gdd', gdd), 
        ('dftoarr', dfta), 
      #  ('pca', pca), 
        ('logistic', logistic)
    ])

In [24]:
pipe.fit_transform(X, y.values)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[  1.,   1.,   0.,   0.,   1.],
       [  2.,   2.,   2.,   2.,   2.],
       [ 23.,  23.,  30.,  21.,  23.],
       ..., 
       [  0.,   0.,  17.,  13.,  13.],
       [  0.,   0.,  17.,  17.,  17.],
       [  0.,   0.,   2.,   2.,   2.]])